In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict,Annotated
from langchain_core.messages import HumanMessage, BaseMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

In [3]:
class ChatState(TypedDict):
    
    messages: Annotated[list[BaseMessage],add_messages]

In [ ]:
import os

# Set your OpenAI API key here (replace with your actual key)
os.environ['OPENAI_API_KEY'] ='your openai api key here'

In [5]:
llm = ChatOpenAI()

def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']
    
    # send to llm
    response = llm.invoke(messages)
    
    # response store state
    return {'messages': [response]}

In [6]:
checkpointer = MemorySaver()

graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)

In [7]:


config = {'configurable': {'thread_id': '1'}}

In [8]:
initial_state = {'messages': [HumanMessage(content="Hello! How are you?")]}

response = chatbot.invoke(initial_state, config=config)
print(response)

{'messages': [HumanMessage(content='Hello! How are you?', additional_kwargs={}, response_metadata={}, id='757804a4-49c0-4c4b-b267-73e64f11af42'), AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 13, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CxVUIAnk8DYnra6QWouaWZjxdV3Xf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b412922b-fc88-4fac-86fd-05b74b37b3fa-0', usage_metadata={'input_tokens': 13, 'output_tokens': 31, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_t

In [9]:
thread_id = '1'

while True:

    input_text = input("Type here: ")

    print('User :', input_text)

    if input_text.strip().lower() in ['exit', 'quit','bye']:
        print("Exiting the chatbot. Goodbye!")
        break       
    
    config ={'configurable':{'thread_id':thread_id}}
    response = chatbot.invoke({'messages': [HumanMessage(content=input_text)]}, config=config)

    print('AI :', response['messages'][-1].content)

User : hi how can you help me
AI : I can assist you with a variety of things! If you have any questions, need information on a certain topic, want help with a task, or just want to chat, feel free to ask me anything. I'm here to help in any way I can.
User : what is the capital of india
AI : The capital of India is New Delhi.
User : exit
Exiting the chatbot. Goodbye!


In [10]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='Hello! How are you?', additional_kwargs={}, response_metadata={}, id='757804a4-49c0-4c4b-b267-73e64f11af42'), AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 13, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CxVUIAnk8DYnra6QWouaWZjxdV3Xf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b412922b-fc88-4fac-86fd-05b74b37b3fa-0', usage_metadata={'input_tokens': 13, 'output_tokens': 31, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache